In [17]:
import pandas as pd
import numpy as np
from pathlib import Path
from fastai.text import *

In [18]:
DATA_PATH = Path('data')
DATA_PATH

PosixPath('data')

In [19]:
TMP_PATH = DATA_PATH/'tmp'
TMP_PATH.mkdir(exist_ok=True)

## Data

In [22]:
dataset = pd.read_json(DATA_PATH/'train.json')
dataset.head()

,comments,description,id,sql,sql_plain,title,url
0,[],,586912,"SELECT p.Title,p.score,p.ViewCount,p.AnswerCou...","SELECT \np.Title,p.score,p.ViewCount,p.AnswerC...",Repent 151-160,http://data.stackexchange.com/stackoverflow/re...
1,[This is test data for my question to make thi...,,54791,"CREATE TABLE #CustomerDB ( ID int NOT NULL, NA...",-- This is test data for my question to make t...,This is test data for my question to make thin...,http://data.stackexchange.com/stackoverflow/re...
2,[],,611565,select Id from Comments where lower(Text) like...,select Id\nfrom Comments \nwhere lower(Text) l...,Accept answer,http://data.stackexchange.com/stackoverflow/re...
3,[],,612812,"DECLARE @From DATETIME = convert(DATETIME, '##...","DECLARE @From DATETIME = convert(DATETIME, '##...",Questions with most views created within 3 mon...,http://data.stackexchange.com/stackoverflow/re...
4,"[Most Prolific Answerers (with score), Shows t...",,325114,"SELECT TOP 50 OwnerUserId as [User Link], COUN...",-- Most Prolific Answerers (with score)\n-- Sh...,Most Prolific Answers,http://data.stackexchange.com/stackoverflow/re...


In [23]:
SQL = dataset['sql']

In [24]:
def fixup(x):
    for bereplaced, replace in token_replaced.items():        
        x = x.replace(bereplaced, replace)
    return x

In [25]:
## Transform the text to be tokenize readily, as certain token has special meaning in SQL, ie. "." means the schema relation
token_replaced = {',' : ' , ',
                 '#' : ' # ',
                 '@' : ' @ ',
                 '!' : ' ! ',
                 '.' : ' . ',
                 '%' : ' % ',
                 '?' : ' ? ',
                 ')' : ' ) ',
                 '(' : ' ( ',
                 '=' : ' = ',
                 '/' : ' / ',
                 '*' : ' * '}
    


In [26]:
SQL = SQL.apply(fixup)

In [27]:
## Sanity Check
SQL.iloc[0]

'SELECT p . Title , p . score , p . ViewCount , p . AnswerCount , p . CommentCount , LEN ( p . body )  as ques_size ,  p . favoritecount , p . id ,  u . reputation as user_repo , u . creationdate as user_join_date ,  datediff ( MINUTE ,  p . CreationDate ,  a . CreationDate )  as QATimeGap from posts as p ,  posts as a ,  users as u where p . id = a . parentId and p . owneruserid = u . id and a . creationdate  =   ( select min ( tau . creationdate )  from posts tau where tau . parentID = p . id group by tau . parentID )  and  (  p . title like \' % Recommended method for handling UnsupportedEncodingException from String . getBytes ( "UTF-8" )  % \' or p . title like \' % Way to format strings with " ? " parameters to full string in java ?  % \' or p . title like \' % How to parse a string without regular expressions % \' or p . title like \' % How to replace string only once without regex in Java ?  % \' or p . title like \' % Efficiently removing specific characters  ( some punctuatio

In [28]:
SQL.iloc[1]

"CREATE TABLE  # CustomerDB  (  ID int NOT NULL ,  NAME nvarchar ( 50 )  NULL ,  COMPANY nvarchar ( 50 )  NULL ,  ADDR1 nvarchar ( 50 )  NULL ,   )  INSERT INTO  # CustomerDB ( ID ,  NAME ,  COMPANY ,  ADDR1 )  VALUES  ( 16242 , 'TOM E' , 'Company A' , '101 First RD' )  ,   ( 16241 , 'RONALD J' , 'Company B' , '12 Tenth AVE' )  ,   ( 16235 , 'KENNETH H' , '' , '12 Tenth AVE' )  ,   ( 16238 , 'MICHAEL H' , 'Company C' , '12 Tenth AVE' )  ,   ( 16243 , 'ANTONIO D' , 'Company D' , '264 Long ST STE 5' )  ,   ( 16237 , 'MICHAEL B' , 'Company E' , '264 Long ST STE 5' )  ,   ( 16234 , 'WALTER L' , 'Company F' , '73 North RD' )  ,   ( 16236 , 'CARL O' , 'Company G' , '73 North RD' )  ,   ( 16239 , 'MICHAEL S' , 'Company H' , '73 North RD' )  ,   ( 16240 , 'MICHAEL I' , 'Company I' , '73 North RD' )  SELECT c1 . id ,  c1 . name ,  c1 . company ,  c1 . addr1 FROM  # CustomerDB c1 LEFT OUTER JOIN  # CustomerDB c2 ON  ( c1 . addr1 =  c2 . addr1 )  GROUP BY c1 . addr1 ,  c1 . id ,  c1 . name ,  c1 

In [29]:
SQL.iloc[100]

"select a . month ,  total ,  unanswered ,  cast ( cast ( unanswered * 1 . 0 / total * 100 as decimal ( 5 , 1 )  )  as varchar ( 5 )  )  + ' % ' as PercentUnanswered from  ( select Convert ( varchar ( 6 )  ,  creationdate ,  112 )  as month ,  count (  *  )  as total from posts where posttypeid  =  1 group by Convert ( varchar ( 6 )  ,  creationdate ,  112 )  )  a ,   ( select Convert ( varchar ( 6 )  ,  creationdate ,  112 )  as month ,  count (  *  )  as unanswered from posts where postTypeID  =  1 and id not in  (  select distinct ParentId from posts where PostTypeId  =  2 )  group by Convert ( varchar ( 6 )  ,  creationdate ,  112 )  )  b where a . month  =  b . month order by a . month"

In [ ]:
sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL)) 
# sql_tok = Tokenizer.proc_all_mp(partition_by_cores(SQL.iloc[:10])) # For Testing

In [158]:
tmp[0]

[' ',
 't_up',
 'select',
 'p',
 '.',
 'title',
 ',',
 'p',
 '.',
 'score',
 ',',
 'p',
 '.',
 'viewcount',
 ',',
 'p',
 '.',
 'answercount',
 ',',
 'p',
 '.',
 'commentcount',
 ',',
 't_up',
 'len',
 '(',
 'p',
 '.',
 'body',
 ')',
 'as',
 'ques_size',
 ',',
 'p',
 '.',
 'favoritecount',
 ',',
 'p',
 '.',
 'i',
 'd',
 ',',
 'u',
 '.',
 'reputation',
 'as',
 'user_repo',
 ',',
 'u',
 '.',
 'creationdate',
 'as',
 'user_join_date',
 ',',
 'datediff',
 '(',
 't_up',
 'minute',
 ',',
 'p',
 '.',
 'creationdate',
 ',',
 'a',
 '.',
 'creationdate',
 ')',
 'as',
 'qatimegap',
 'from',
 'posts',
 'as',
 'p',
 ',',
 'posts',
 'as',
 'a',
 ',',
 'users',
 'as',
 'u',
 'where',
 'p',
 '.',
 'i',
 'd',
 '=',
 'a',
 '.',
 'parentid',
 'and',
 'p',
 '.',
 'owneruserid',
 '=',
 'u',
 '.',
 'i',
 'd',
 'and',
 'a',
 '.',
 'creationdate',
 '=',
 '(',
 'select',
 'min',
 '(',
 'tau',
 '.',
 'creationdate',
 ')',
 'from',
 'posts',
 'tau',
 'where',
 'tau',
 '.',
 'parentid',
 '=',
 'p',
 '.',
 'i',
 'd

## Tokenize

In [16]:
np.percentile([len(o) for o in sql_tok], 90)

NameError: name 'sql_tok' is not defined

In [17]:
keep = np.array([len(o)<30 for o in sql_tok])

NameError: name 'sql_tok' is not defined

In [164]:
sql_tok = np.array(sql_tok)[keep]

In [167]:
# pickle.dump(sql_tok, (DATA_PATH/'SQL_tok.pkl').open('wb'))

In [7]:
sql_tok = pickle.load((DATA_PATH/'sql_tok.pkl').open('rb'))

NameError: name 'pickle' is not defined

In [19]:
sql_tok.shape

(5081,)

In [13]:
def toks2ids(tok,pre):
    freq = Counter(p for o in tok for p in o)
    itos = [o for o,c in freq.most_common(40000)]
    itos.insert(0, '_bos_')
    itos.insert(1, '_pad_')
    itos.insert(2, '_eos_')
    itos.insert(3, '_unk')
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    ids = np.array([([0] + [stoi[o] for o in p] + [2]) for p in tok])
    np.save(TMP_PATH/f'{pre}_ids.npy', ids)
    pickle.dump(itos, open(TMP_PATH/f'{pre}_itos.pkl', 'wb'))
    return ids,itos,stoi

In [64]:
# sql_ids,sql_itos,sql_stoi = toks2ids(sql_tok,'sql')


In [14]:
def load_ids(pre):
    ids = np.load(TMP_PATH/f'{pre}_ids.npy')
    itos = pickle.load(open(TMP_PATH/f'{pre}_itos.pkl', 'rb'))
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
    return ids,itos,stoi

In [15]:
sql_ids,sql_itos,sql_stoi = load_ids('sql')


In [16]:
[sql_itos[o] for o in sql_ids[0]], len(sql_itos)

(['_bos_',
  '_bos_',
  ' ',
  't_up',
  'select',
  'p',
  '.',
  'title',
  ',',
  'p',
  '.',
  'score',
  ',',
  'p',
  '.',
  'viewcount',
  ',',
  'p',
  '.',
  'answercount',
  ',',
  'p',
  '.',
  'commentcount',
  ',',
  't_up',
  'len',
  '(',
  'p',
  '.',
  'body',
  ')',
  'as',
  'ques_size',
  ',',
  'p',
  '.',
  'favoritecount',
  ',',
  'p',
  '.',
  'i',
  'd',
  ',',
  'u',
  '.',
  'reputation',
  'as',
  'user_repo',
  ',',
  'u',
  '.',
  'creationdate',
  'as',
  'user_join_date',
  ',',
  'datediff',
  '(',
  't_up',
  'minute',
  ',',
  'p',
  '.',
  'creationdate',
  ',',
  'a',
  '.',
  'creationdate',
  ')',
  'as',
  'qatimegap',
  'from',
  'posts',
  'as',
  'p',
  ',',
  'posts',
  'as',
  'a',
  ',',
  'users',
  'as',
  'u',
  'where',
  'p',
  '.',
  'i',
  'd',
  '=',
  'a',
  '.',
  'parentid',
  'and',
  'p',
  '.',
  'owneruserid',
  '=',
  'u',
  '.',
  'i',
  'd',
  'and',
  'a',
  '.',
  'creationdate',
  '=',
  '(',
  'select',
  'min',
  '('

## Dataset

In [86]:
sql_ids_tr = np.array([o[:sqllen_90] for o in sql_ids])

In [163]:
cs=8

In [164]:
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

NameError: name 'idx' is not defined

In [165]:
xs = np.stack(c_in_dat, axis=0)

NameError: name 'c_in_dat' is not defined

In [160]:
val_idx = get_cv_idxs(len(idx)-cs-1)
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

NameError: name 'idx' is not defined

In [88]:
np.random.seed(42)
trn_keep = np.random.rand(len(sql_ids_tr))>0.1
sql_trn,sql_trn = sql_ids_tr[trn_keep],sql_ids_tr[trn_keep]
sql_val,sql_val = sql_ids_tr[~trn_keep],sql_ids_tr[~trn_keep]
len(sql_trn),len(sql_val)

(4548, 533)

In [91]:
trn_ds = Seq2SeqDataset(sql_trn,sql_trn)
val_ds = Seq2SeqDataset(sql_val,sql_val)

In [92]:
bs=128
trn_samp = SortishSampler(sql_trn, key=lambda x: len(sql_trn[x]), bs=bs)
val_samp = SortSampler(sql_val, key=lambda x: len(sql_val[x]))

In [95]:
trn_dl = DataLoader(trn_ds, bs, transpose=True, transpose_y=True, num_workers=1, 
                    pad_idx=1, pre_pad=False, sampler=trn_samp)
val_dl = DataLoader(val_ds, int(bs*1.6), transpose=True, transpose_y=True, num_workers=1, 
                    pad_idx=1, pre_pad=False, sampler=val_samp)
md = ModelData(DATA_PATH,trn_dl, val_dl)

In [96]:
it = iter(trn_dl)
its = [next(it) for i in range(5)]
[(len(x),len(y)) for x,y in its]

C:\Users\NokChan\Anaconda3\envs\fastai\lib\site-packages\torch\cuda\__init__.py:116: UserWarning: 
    Found GPU0 GeForce 930M which is of cuda capability 5.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


[(30, 30), (14, 14), (28, 28), (15, 15), (30, 30)]

## Model

In [69]:
def create_emb(itos, em_sz):
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)
    return emb

In [98]:
dim_sql_vec

100

In [107]:
len(sql_itos)

2342

In [108]:
dim_sql_vec = 100 # HardCode
n_hidden = 64 # size of hidden layer of neurons
learning_rate = 1e-1

In [139]:
import pdb

In [153]:
class SeqSQLModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, dim_sql_vec, padding_idx=1)
        self.l_in = nn.Linear(dim_sql_vec+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            print('h:',h.size())
            print('c:',c.size())
            print(self.e(c).size())
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [154]:
m = SeqSQLModel(len(sql_itos),dim_sql_vec).cuda()

In [155]:
opt = optim.Adam(m.parameters(), 1e-3)

In [156]:
it = iter(md.trn_dl)

In [158]:
*xs,yt = next(it)

In [159]:
m(*V(xs))

h: torch.Size([30, 64])
c: torch.Size([30, 128])
torch.Size([30, 128, 100])


RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 2 at c:\pytorch\torch\lib\thc\generic/THCTensorMath.cu:102

In [124]:
t = m(*V(xs))

TypeError: torch.index_select received an invalid combination of arguments - got (torch.FloatTensor, int, !torch.cuda.LongTensor!), but expected (torch.FloatTensor source, int dim, torch.LongTensor index)

In [74]:
dim_sql_vec = 100
nh = 256

In [78]:
sqllen_90 = int(np.percentile([len(o) for o in en_ids], 99))
sqllen_90

30

In [79]:
rnn = Word(sql_itos, dim_sql_vec, nh, sqllen_90)
learn = RNN_Learner(md, SingleModel(to_gpu(rnn)), opt_fn=opt_fn)
learn.crit = seq2seq_loss

NameError: name 'md' is not defined

In [ ]:
learn.lr_find()
learn.sched.plot()

## Inference

In [ ]:
x,y = next(iter(val_dl))
probs = learn.model(V(x))
preds = to_np(probs.max(2)[1])

for i in range(180,190):
    print(' '.join([fr_itos[o] for o in x[:,i] if o != 1]))
    print(' '.join([en_itos[o] for o in y[:,i] if o != 1]))
    print(' '.join([en_itos[o] for o in preds[:,i] if o!=1]))
    print()

##  State Machine

## Generate text base on Inference and state machine